In [ ]:
# for colab
# !pip install spacy_stanza
# !pip install ckip_transformers

# 1. import packages

In [7]:
import pandas as pd
from tqdm.notebook import tqdm
import pickle
import pathlib
import sys
from spacy.tokens import Doc

# 2. Prepare Spacy Pipeline

In [8]:
spacy_pipeline_parent_path = pathlib.Path.cwd().parent.parent.parent.parent
sys.path.append(str(spacy_pipeline_parent_path))

from spacy_pipeline import pipeline_setup
from spacy_pipeline import opinion_rule

In [10]:
methods = {
    "opinion_v0": {
        "version": "opinion_v0",
        "pattern": [
            {
                "RIGHT_ID": "OPINION_OPR_found_root",
                "RIGHT_ATTRS": {
                    "TAG": {
                        "IN": ["VE"]
                    },
                }
            },
            {
                "LEFT_ID": "OPINION_OPR_found_root",
                "REL_OP": ">",
                "RIGHT_ID": "OPINION_SRC_found_root",
                "RIGHT_ATTRS": {
                    "DEP": {
                        "IN": ["nsubj"]
                    },
                }
            },
            {
                "LEFT_ID": "OPINION_OPR_found_root",
                "REL_OP": ">",
                "RIGHT_ID": "OPINION_SEG_found_root",
                "RIGHT_ATTRS": {
                    "DEP": {
                        "IN": ["ccomp"]#, "parataxis"]
                    },
                    # "POS": {
                    #         "IN": ["VERB", "NOUN", "ADJ"]
                    # }
                }
            }
        ]
    },
}

In [ ]:
pipeline = pipeline_setup.get_opinion_pipeline(methods['opinion_v0'])
vocab = pipeline.vocab

## 2.2 Define workflow

In [ ]:
def run_work_flow(all_docs, spacy_pipeline, n_process=1):
    
    # try:
    with tqdm(total=len(all_docs)) as pbar:
        
        for paragraphs in all_docs:
                
            for i, doc in enumerate(spacy_pipeline.pipe(paragraphs, n_process=n_process)):
                pass
            pbar.update(1)
    return all_docs

# load all_docs

In [ ]:
pickle_dir = str(pathlib.Path.cwd().parent)
pickle_file = 'label_news_200_docs_stanza.pkl'

with open(pickle_dir + '/' + pickle_file, 'rb') as f:
    bytes_data = pickle.load(f)
    all_docs_dev = [[Doc(vocab).from_bytes(doc_bytes) for doc_bytes in docs] for docs in bytes_data]

all_docs_dev = run_work_flow(all_docs_dev, pipeline, n_process=1)

# load all_docs_test

In [ ]:
pickle_dir = str(pathlib.Path.cwd().parent)
pickle_file = 'label_news_50_test_docs_stanza.pkl'

with open(pickle_dir + '/' + pickle_file, 'rb') as f:
    bytes_data = pickle.load(f)
    all_docs_test = [[Doc(vocab).from_bytes(doc_bytes) for doc_bytes in docs] for docs in bytes_data]

all_docs_test = run_work_flow(all_docs_test, pipeline, n_process=1)

# span level evaluation

In [ ]:
def get_precision(TP, FP):
    return TP / (TP + FP)

def get_recall(TP, FN):
    return TP / (TP + FN)

def get_f_score(precision, recall, beta=1):
    return (1 + beta**2) * precision * recall / (beta**2 * precision + recall)

label_match_dict = {
    "OPINION_SRC": "OPINION_SRC_found",
    "OPINION_OPR": "OPINION_OPR_found",
    "OPINION_SEG": "OPINION_SEG_found" 
}

def delete_around_punt_check_span(label_span, match_span):
    puct_list = ['，', '。', '「', '」', ' ', '！', '？', ';', ':', "'", '"', '‘', '“', '『', '』', '、', '（', '）']
    
    while label_span.text[0] in puct_list:
        label_span = label_span[1:]

    while label_span.text[-1] in puct_list:
        label_span = label_span[:-1] 
    
    while match_span.text[0] in puct_list:
        match_span = match_span[1:]

    while match_span.text[-1] in puct_list:
        match_span = match_span[:-1]

    
    if label_span.start == match_span.start and label_span.end == match_span.end:
        return True
    else:
        return False
    
def interval_check_span(label_span, match_span):
    if label_span.start <= match_span.start and label_span.end >= match_span.end:
        return True
    else:
        return False
    
def tolerance_check_span(label_span, match_span, tolerance=1):
    if abs(label_span.start - match_span.start) <= tolerance and label_span.end == match_span.end:
        return True
    if label_span.start == match_span.start and abs(label_span.end - match_span.end) <= tolerance:
        return True
    else:
        return False

In [ ]:
def new_eval(all_docs, method):
    
    counter = {
        'OPINION_OPR_FP': 0,
        'OPINION_OPR_FN': 0,
        'OPINION_OPR_TP': 0,

        'OPINION_SRC_FP': 0,
        'OPINION_SRC_FN': 0,
        'OPINION_SRC_TP': 0,
        
        'OPINION_SEG_FP': 0,
        'OPINION_SEG_FN': 0,
        'OPINION_SEG_TP': 0,
    }

    for docs in all_docs:
        for doc in docs:

            spans = {
                'OPINION_SRC_TP': [],
                'OPINION_OPR_TP': [],
                'OPINION_SEG_TP': [],

                'OPINION_SRC_FP': [],
                'OPINION_OPR_FP': [],
                'OPINION_SEG_FP': [],

                'OPINION_SRC_FN': [],
                'OPINION_OPR_FN': [],
                'OPINION_SEG_FN': [],
            }
            
            if 'opinion_found' in doc.spans and 'opinion_label' in doc.spans:
                for found_span in doc.spans['opinion_found']:
                    found_match = False
                    for label_span in doc.spans['opinion_label']:
                        if method(label_span, found_span) and found_span.label_ == label_match_dict[label_span.label_]:
                            spans[f"{label_span.label_}_TP"].append(label_span)
                            found_match = True
                            break
                    if not found_match:
                        spans[f"{found_span.label_[:11]}_FP"].append(found_span)
                
                for label_span in doc.spans['opinion_label']:
                    if label_span not in spans[f"{label_span.label_}_TP"]:
                        spans[f"{label_span.label_}_FN"].append(label_span)

            elif 'opinion_found' in doc.spans:
                for found_span in doc.spans['opinion_found']:
                    spans[f"{found_span.label_[:11]}_FP"].append(found_span)

            elif 'opinion_label' in doc.spans:
                for label_span in doc.spans['opinion_label']:
                    spans[f"{label_span.label_}_FN"].append(label_span)

            for key in counter.keys():
                counter[key] += len(spans[key])

    return {
        'OPINION_SRC': {
            'precision': get_precision(counter['OPINION_SRC_TP'], counter['OPINION_SRC_FP']),
            'recall': get_recall(counter['OPINION_SRC_TP'], counter['OPINION_SRC_FN']),
            'f_score': get_f_score(get_precision(counter['OPINION_SRC_TP'], counter['OPINION_SRC_FP']), get_recall(counter['OPINION_SRC_TP'], counter['OPINION_SRC_FN'])),
        },
        'OPINION_OPR': {
            'precision': get_precision(counter['OPINION_OPR_TP'], counter['OPINION_OPR_FP']),
            'recall': get_recall(counter['OPINION_OPR_TP'], counter['OPINION_OPR_FN']),
            'f_score': get_f_score(get_precision(counter['OPINION_OPR_TP'], counter['OPINION_OPR_FP']), get_recall(counter['OPINION_OPR_TP'], counter['OPINION_OPR_FN'])),
        },
        'OPINION_SEG': {
            'precision': get_precision(counter['OPINION_SEG_TP'], counter['OPINION_SEG_FP']),
            'recall': get_recall(counter['OPINION_SEG_TP'], counter['OPINION_SEG_FN']),
            'f_score': get_f_score(get_precision(counter['OPINION_SEG_TP'], counter['OPINION_SEG_FP']), get_recall(counter['OPINION_SEG_TP'], counter['OPINION_SEG_FN']))
        },
        'counter': counter
    }

In [ ]:
new_eval(all_docs_dev, delete_around_punt_check_span)

In [ ]:
new_eval(all_docs_test, delete_around_punt_check_span)

In [ ]:
len(all_docs_test)

In [ ]:
# new_eval(all_docs_dev, delete_around_punt_check_span)
# new_eval(all_docs, interval_check)
# new_eval(all_docs, tolerance_check)

In [ ]:
# new_eval(all_docs_test, delete_around_punt_check_span)

# token level evaluation

In [ ]:
def eval_token_level(all_docs):

    counter = {
        'OPINION_SRC_FP': 0,
        'OPINION_OPR_FP': 0,
        'OPINION_SEG_FP': 0,

        'OPINION_SRC_FN': 0,
        'OPINION_OPR_FN': 0,
        'OPINION_SEG_FN': 0,
        
        'OPINION_SRC_TP': 0,
        'OPINION_OPR_TP': 0,
        'OPINION_SEG_TP': 0,
    }

    for docs in all_docs:
        for doc in docs:
            
            if 'opinion_found' in doc.spans and 'opinion_label' in doc.spans:
                for token in doc:
                    if token._.found_type == "OPINION_SRC_found":
                        if "OPINION_SRC" in token._.label_type:
                            counter['OPINION_SRC_TP'] += 1
                        else:
                            counter['OPINION_SRC_FP'] += 1
                    elif token._.found_type == "OPINION_OPR_found":
                        if "OPINION_OPR" in token._.label_type:
                            counter['OPINION_OPR_TP'] += 1
                        else:
                            counter['OPINION_OPR_FP'] += 1
                    elif token._.found_type == "OPINION_SEG_found":
                        if "OPINION_SEG" in token._.label_type:
                            counter['OPINION_SEG_TP'] += 1
                        else:
                            counter['OPINION_SEG_FP'] += 1
                
                for token in doc:
                    if "OPINION_SRC" in token._.label_type and token._.found_type != "OPINION_SRC_found":
                        counter['OPINION_SRC_FN'] += 1
                    elif "OPINION_OPR" in token._.label_type and token._.found_type != "OPINION_OPR_found":
                        counter['OPINION_OPR_FN'] += 1
                    elif "OPINION_SEG" in token._.label_type and token._.found_type != "OPINION_SEG_found":
                        counter['OPINION_SEG_FN'] += 1

    return {
        'OPINION_SRC': {
            'precision': get_precision(counter['OPINION_SRC_TP'], counter['OPINION_SRC_FP']),
            'recall': get_recall(counter['OPINION_SRC_TP'], counter['OPINION_SRC_FN']),
            'f_score': get_f_score(get_precision(counter['OPINION_SRC_TP'], counter['OPINION_SRC_FP']), get_recall(counter['OPINION_SRC_TP'], counter['OPINION_SRC_FN'])),
        },
        'OPINION_OPR': {
            'precision': get_precision(counter['OPINION_OPR_TP'], counter['OPINION_OPR_FP']),
            'recall': get_recall(counter['OPINION_OPR_TP'], counter['OPINION_OPR_FN']),
            'f_score': get_f_score(get_precision(counter['OPINION_OPR_TP'], counter['OPINION_OPR_FP']), get_recall(counter['OPINION_OPR_TP'], counter['OPINION_OPR_FN'])),
        },
        'OPINION_SEG': {
            'precision': get_precision(counter['OPINION_SEG_TP'], counter['OPINION_SEG_FP']),
            'recall': get_recall(counter['OPINION_SEG_TP'], counter['OPINION_SEG_FN']),
            'f_score': get_f_score(get_precision(counter['OPINION_SEG_TP'], counter['OPINION_SEG_FP']), get_recall(counter['OPINION_SEG_TP'], counter['OPINION_SEG_FN']))
        },
        'counter': counter
    }

In [ ]:
eval_token_level(all_docs_dev)

In [ ]:
eval_token_level(all_docs_test)

In [ ]:
# eval_token_level(all_docs_dev)

In [ ]:
# eval_token_level(all_docs_test)

In [ ]:
all_docs_dev[0][0].ents

In [ ]:
for ent in all_docs_dev[0][0].ents:
    print(ent, ent.label_, ent.start, ent.end)

In [ ]:
docs = all_docs_dev[0]

In [ ]:
ent_list_list = []

for doc in docs:
    ent_list = [ent for ent in doc.ents if ent.label_ == "PERSON"]
    ent_list_list.append(ent_list)

ent_list_list

In [ ]:
s1 = "中國國民黨主席朱立倫今天上午陪國民黨桃園市長候選人張善政車隊掃街時表示，他對張善政有100％信心，能力、學歷與經歷都是最好的桃園市長人選。"
s2 = "對於傳出衛福部高層要求官員路過陳時中的選前之夜幫忙衝人氣，蔣萬安表示，台北未來能夠站上國際舞台、跟國際一流城市並駕齊驅，相互競爭，才是他真正關心的事情。"
s3 = "陳時中在介紹政策口袋書時開玩笑說：「這可以保存30年沒問題，所以要的人很多。」保存30年的哏是前陣子藍白陣營不斷抹黑武漢肺炎（新型冠狀病毒病，COVID-19）疫苗採購價格的公文要「封存」30年，但實際上是要讓公文「保存」30年，讓之後有需要查閱時可獲取。疫苗採購的保密年限不同品牌有不同時間，例如國產高端疫苗要5年（已由高端公司自行公布）、Novavax疫苗要7年、BNT疫苗要10年，屆期馬上可以解密。"
s4 = "林耕仁並承諾老人年金一定會續發，而且65歲以上的長輩，健保費由政府來承擔，更會研擬放寬發放的年限、加碼重陽敬老金從2000到3000，還有擴大敬老卡的使用範圍，並且研擬將額度從600提升到1200。"

In [ ]:
from spacy import displacy


In [ ]:
doc = pipeline(s1)
displacy.render(doc, style="dep", options={'fine_grained':"True"})

In [ ]:
doc = pipeline(s1)
print()
displacy.render(doc, style="span", options={'spans_key':"opinion_found[1]"})

In [1]:
doc1 = ["「炎上 王世堅」上周末登場，前台南市議員謝龍介也應邀出席，沒想到博恩現場大爆謝龍介兒子持毒，大酸「這個爸爸教育多失敗啊！兒子連藏個毒品都不會」，讓謝龍介當場臉超臭。不少網友看完紛紛嚇傻表示「還真的不知道有這事。」對此，謝龍介今（12日）受訪首度證實博恩「脫稿演出」，並親吐「當場臉垮」的內幕！",
            "根據《中時新聞網》採訪，謝龍介透露《炎上》的腳本都是用國語寫的，但我用台語跟觀眾互動會比較溜，他表示現場觀眾都是買票進場，所以先前從中午一直彩排到晚上八點，要做一些準備，這樣演出才會比較自然，跟觀眾的互動才會比較好。不過針對博恩挖出他兒子2013年吸毒事件，讓他當場臉垮掉，對此謝龍介首度證實「事先的稿子完全沒有這段，他們也是脫稿演出，當下有一點驚訝，不過事情過去了就還好，不會影響我在現場的演出，之後如果有機會可能還會再上炎上。」",
            "謝龍介上週末出席「炎上 王世堅」，沒想到博恩提到相當敏感的「兒子持K他命」一事，「蠻羨慕你台語講很好，吸毒台語怎麼講？『恁囝嗎？』這個爸爸教育多失敗，連藏個毒品都不會，有這麼難嗎？你不是整天玩布袋戲，你兒子怎麼不知道往X眼裡塞。」許多網友看完後紛紛嚇傻表示「現場看有看到第一次提到兒子持毒事件時，謝龍介議員臉超臭！有心驚到，後面再提表情稍微笑笑的」、「提到兒子時，謝龍介的臉直接垮」。 ",
            "★ 三立新聞網提醒您：",
            "　莫逞一時樂，遺害百年身！　拒絕毒品　珍惜生命　健康無價　不容毒噬"]

doc2 = [
                "副總統賴清德今天晚間出席台北場「向黨員報告」發表會，他強調，民進黨是第一個土生土長的本土政權，存在目的只有一個，就是為台灣打拚。現在是民進黨新使命的開始，就是守護台灣，未來選舉，我們絕對是直接面對共產黨，所以一定要團結，得到台灣社會絕大多數的支持，力量才會夠。",
            "賴清德今晚在劍潭青年活動中心1樓集賢廳舉行台北市「向黨員報告-政見發表會」，立委吳思瑤、何志偉、高嘉瑜、吳玉琴及多位台北市議員到場支持。",
            "由於何志偉和高嘉瑜近來對民進黨多所批評，高嘉瑜一進場就被支持者大噓「背骨」「下台」，何志偉致詞時也不停被台下的喊叫聲打斷，大嗆「不要官腔官調」「為什麼叫蘇貞昌下台？」何則回應「我沒有叫蘇貞昌下台」。",
            "也有支持者要兩人「加油」，還趨前要求握手，賴清德進場時，被問到如何看待高嘉瑜被嗆聲、是否會幫她抱屈？賴清德面對提問並未回應。",
            "賴清德致詞時表示，雖然大家都不願意看到九合一敗選結果，但既然結果出來就要接受，從失敗中找到原因，重振旗鼓、重新出發。他從新北走到全台各地再到台北市，發現民進黨支持者的力量，「今天如果不是想要繼續和民進黨一起打拚，守護台灣，大家不會在這裡」。",
            "他強調，雖說「敗軍之將不可言勇」，但也有句俗語說「不以勝敗論英雄」，雖然參選台北市長失敗的陳時中不在現場，但他要求現場民眾報以掌聲，肯定陳時中的表現。",
            "現場有很多資深黨員，也有年輕黨員，賴清德指出，這代表無論是哪個年代的人，對民進黨一樣支持，他以當年創黨時的艱辛為例，強調當時的創黨黨員，很多都是抱著犧牲奉獻的決心，來反抗當時獨裁的國民黨，民進黨是台灣第一個土生土長的本土政權，有它必要的使命，存在目的只有一個，就是為台灣打拚。",
            "他強調，如果沒有這種決心和覺悟的人，請他早早離開，我們的意志就是為台灣打拚，沒有其他第二句話。民進黨過去多年，結合台灣社會力量，打破黨禁、報禁，追求百分之百民主自由，推動國會全面改選、總統直選，帶動台灣民主進步和國家發展，民進黨支持者講話敢大聲，大家也都願意犧牲。",
            "賴清德要求大家，現在應該要有這樣的精神，重新喚回社會的感動和支持，過去為台灣民主很多人被關甚至犧生性命，為民主對抗獨裁的國民黨，現在是新民主的起點，也是新時代、新使命的開始，這個使命就是守護台灣。",
            "他說，看看過去的選舉歷史，2000年陳水扁當選，是因為國民黨腐敗，2004年又當選，是因為經過4年執政，認真打拚才贏了一些，2008年民進黨輸，是因為力量不夠，對抗的是國民黨加上共產黨，2012年仍然力量不夠，2016年能贏，是因為國民黨站前面，共產黨站後面。",
            "賴清德強調，未來我們要面對的，要守護台灣，絕對是直接面對共產黨。所以我們一定要團結，得到台灣社會絕大多數的支持，如果能如此，我們的力量就一定夠，如果不團結，力量一定不夠。",
            "他說，台灣的力量「取之不盡，用之不竭」，只要有使命，有力量，方法正確，他相信一定能找回社會的感動，呼籲大家共同打拚，成功守護台灣，為守護台灣共同努力。"
]

doc3 = [
                "〔記者鍾麗華／台北報導〕行政院長蘇貞昌今天（12日）接見立陶宛國會國安及國防事務議員訪團，他致詞時表示，中國的威權擴張毫無道理且蠻橫，台灣應與立陶宛加強合作，一起努力對抗集權擴張行為。",
            "蘇揆強調，立陶宛與台灣都在威權擴張侵略的最前線，兩國同樣堅持自由、民主，有相同的價值。台灣受到中國軍事騷擾和假訊息滲透，訪團成員、立陶宛國會友台小組副主席莎卡琳恩因關切新疆人權問題，被中國制裁，台灣與立陶宛應加強合作對抗蠻橫的極權擴張行為。",
            "蘇揆指出，威權擴張侵略時常違反世界期待，如俄羅斯侵略烏克蘭，透過武力遂行擴張目的，除讓民眾變成難民，也影響整個世界的經濟活動；而中國對台灣的軍事騷擾及各種假訊息滲透，也都是違背世界期待、侵擾區域和平。",
            "卡斯楚那斯表示，民主國家正努力對抗威權擴張，未來可能會有衝突，這次與台灣官員、智庫交流有個共識，就是國防不能只靠軍隊，應該要靠社會一起執行全民國防，他也願意分享立陶宛的經驗。立陶宛很有韌性，面對中國壓力還是存活，未來希望扮演歐洲國家的典範，與台灣成功合作，而這次訪台在投資、技術合作方面都有好消息，希望未來可以進一步落實。",
            "立陶宛國會友台小組副主席莎卡琳恩指出，台灣與立陶宛都距離敵人很近，這同時是威脅也是優勢，那就是可以近距離觀察敵人，越來越多人理解到中國的危險，立陶宛也在歐洲努力倡導這個概念，如同俄羅斯侵略烏克蘭一樣，如果台灣被入侵，全世界都會受到影響，應該全力避免。"
]

In [56]:
reversed([1, 2, 3])

In [149]:
def get_name_list(docs, paragraph_index):
    ent_list_list = []
    for doc in docs:
        ent_list = [ent for ent in doc.ents if ent.label_ == "PERSON"]
        ent_list.reverse()
        ent_list_list.append(ent_list)
    same_paragraph = ent_list_list[paragraph_index] 
    ent_list_list = ent_list_list[:paragraph_index] 
    ent_list_list.reverse()
    return same_paragraph, [i for j in ent_list_list for i in j]

def check_SRC_type(span):
    if len(span) == 1:
        token = span[0]
        if len(token) == 1:
            if token.pos_ == "PRON":
                return (token.pos_, token.text) # 他
            elif token.pos_ == "PROPN":
                return (token.pos_, token.text)# 張
        # else:
        #     if token.pos_ == "NOUN":
        #         return True # 官員 總統
    
    elif len(span) == 2:
        token1, token2 = span[0], span[1]
        if len(token1) == 1 and len(token2) == 2:
            if token1.pos_ == "PROPN" and token2.pos_ == "NOUN":
                return (token1.pos_, token1.text) # 蔡總統 盧市長
        
        # if token1.pos_ == "NOUN" and token2.pos_ == "NOUN":
        #     return True # 媽媽市長 縣黨部
    
    elif len(span) == 3:
        token1, token2, token3 = span[0], span[1], span[2]
        if len(token1) == 1 and len(token2) == 1 and len(token3) == 2:
            if token1.pos_ == "PROPN" and token2.pos_ == "PART" and token3.pos_ == "NOUN":
                return (token1.pos_, token1.text) # 賴副總統
    
    # if len(span) >= 2:
    #     if span[0].pos_ == "DET" and all(["NOUN" in token.pos_ for token in span[1:]]):
    #         return "DET" # 該綠營人士 這名黨政人士 該人士 該立委
        
    
        
        # elif span[0].pos_ == "NUM" and all(["NOUN" in token.pos_ for token in span[1:]]):
        #     return True # 3位監委 2位監委
    return None

def find_pronoun_resolution(docs, pragraph_index, span):
    span_type= check_SRC_type(span)
    print("span_type", span_type)
    if span_type != None:
        print("span_type[0]", span_type[0], span_type[0] == "PRON")
        if span_type[0] == "PRON":
            names_same_paragraph, names_before = get_name_list(docs, pragraph_index)
            print("here")
            print("PRON", names_same_paragraph, names_before)
            if len(names_same_paragraph) != 0:
                for name in names_same_paragraph:
                    if name.start < span.start:
                        return name
            if len(names_before) > 0:
                return names_before[0]
            return None
        
        if span_type[0] == "PROPN":
            names_same_paragraph, names_before = get_name_list(docs, pragraph_index, span)
            print("PRON", names_same_paragraph, names_before)
            if len(names_same_paragraph) != 0:
                for name in names_same_paragraph:
                    if name.start < span.start and span_type[1] in name.text:
                        return name
            if len(names_before) > 0:
                for name in names_before:
                    if name.start < span.start and span_type[1] in name.text:
                        return name
            return None
        return None
    return None

In [151]:
find_pronoun_resolution(doc1, paragraph_index, target_span)

span_type ('PRON', '他')
span_type[0] PRON True
here
PRON [博恩, 謝龍介今, 謝龍介, 大爆謝龍介, 博恩, 謝龍介, 王世堅] []


大爆謝龍介

In [153]:
paragraph_index

0

In [152]:
doc1[0]

[「炎上 王世堅」上周末登場，前台南市議員謝龍介也應邀出席，沒想到博恩現場大爆謝龍介兒子持毒，大酸「這個爸爸教育多失敗啊！兒子連藏個毒品都不會」，讓謝龍介當場臉超臭。不少網友看完紛紛嚇傻表示「還真的不知道有這事。」對此，謝龍介今（12日）受訪首度證實博恩「脫稿演出」，並親吐「當場臉垮」的內幕！,
 根據《中時新聞網》採訪，謝龍介透露《炎上》的腳本都是用國語寫的，但我用台語跟觀眾互動會比較溜，他表示現場觀眾都是買票進場，所以先前從中午一直彩排到晚上八點，要做一些準備，這樣演出才會比較自然，跟觀眾的互動才會比較好。不過針對博恩挖出他兒子2013年吸毒事件，讓他當場臉垮掉，對此謝龍介首度證實「事先的稿子完全沒有這段，他們也是脫稿演出，當下有一點驚訝，不過事情過去了就還好，不會影響我在現場的演出，之後如果有機會可能還會再上炎上。」,
 謝龍介上週末出席「炎上 王世堅」，沒想到博恩提到相當敏感的「兒子持K他命」一事，「蠻羨慕你台語講很好，吸毒台語怎麼講？『恁囝嗎？』這個爸爸教育多失敗，連藏個毒品都不會，有這麼難嗎？你不是整天玩布袋戲，你兒子怎麼不知道往X眼裡塞。」許多網友看完後紛紛嚇傻表示「現場看有看到第一次提到兒子持毒事件時，謝龍介議員臉超臭！有心驚到，後面再提表情稍微笑笑的」、「提到兒子時，謝龍介的臉直接垮」。 ,
 ★ 三立新聞網提醒您：,
 　莫逞一時樂，遺害百年身！　拒絕毒品　珍惜生命　健康無價　不容毒噬]

In [73]:
len(doc1)

5

In [90]:
for doc in doc2:
    for s

謝龍介
博恩
謝龍介
博恩
謝龍介今
謝龍介
大爆謝龍介
博恩
謝龍介
王世堅


In [78]:
print(get_name_list(doc1, 2))

([謝龍介, 謝龍介, 博恩, 王世堅, 謝龍介上], [謝龍介, 博恩, 謝龍介, 博恩, 謝龍介今, 謝龍介, 大爆謝龍介, 博恩, 謝龍介, 王世堅])


In [80]:
l = get_name_list(doc1, 2)

In [84]:
for i in l[1]:
    print([j.pos_ for j in i])

['PROPN']
['PROPN']
['PROPN']
['PROPN']
['PROPN', 'NOUN']
['PROPN']
['VERB']
['PROPN']
['PROPN']
['PART', 'PROPN']


In [87]:
l[0]

[謝龍介, 謝龍介, 博恩, 王世堅, 謝龍介上]

In [106]:
doc1

[「炎上 王世堅」上周末登場，前台南市議員謝龍介也應邀出席，沒想到博恩現場大爆謝龍介兒子持毒，大酸「這個爸爸教育多失敗啊！兒子連藏個毒品都不會」，讓謝龍介當場臉超臭。不少網友看完紛紛嚇傻表示「還真的不知道有這事。」對此，謝龍介今（12日）受訪首度證實博恩「脫稿演出」，並親吐「當場臉垮」的內幕！,
 根據《中時新聞網》採訪，謝龍介透露《炎上》的腳本都是用國語寫的，但我用台語跟觀眾互動會比較溜，他表示現場觀眾都是買票進場，所以先前從中午一直彩排到晚上八點，要做一些準備，這樣演出才會比較自然，跟觀眾的互動才會比較好。不過針對博恩挖出他兒子2013年吸毒事件，讓他當場臉垮掉，對此謝龍介首度證實「事先的稿子完全沒有這段，他們也是脫稿演出，當下有一點驚訝，不過事情過去了就還好，不會影響我在現場的演出，之後如果有機會可能還會再上炎上。」,
 謝龍介上週末出席「炎上 王世堅」，沒想到博恩提到相當敏感的「兒子持K他命」一事，「蠻羨慕你台語講很好，吸毒台語怎麼講？『恁囝嗎？』這個爸爸教育多失敗，連藏個毒品都不會，有這麼難嗎？你不是整天玩布袋戲，你兒子怎麼不知道往X眼裡塞。」許多網友看完後紛紛嚇傻表示「現場看有看到第一次提到兒子持毒事件時，謝龍介議員臉超臭！有心驚到，後面再提表情稍微笑笑的」、「提到兒子時，謝龍介的臉直接垮」。 ,
 ★ 三立新聞網提醒您：,
 　莫逞一時樂，遺害百年身！　拒絕毒品　珍惜生命　健康無價　不容毒噬]

In [88]:
l[1]

[謝龍介, 博恩, 謝龍介, 博恩, 謝龍介今, 謝龍介, 大爆謝龍介, 博恩, 謝龍介, 王世堅]

In [79]:
print([(i.start, i.end)for i in get_name_list(doc1, 5)])

IndexError: list index out of range

In [11]:
pipeline = pipeline_setup.get_opinion_pipeline(methods['opinion_v0'])
vocab = pipeline.vocab

2023-03-28 18:55:25 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-03-28 18:55:27 INFO: Loading these models for language: zh-hant (Traditional_Chinese):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| depparse  | gsd     |

2023-03-28 18:55:27 INFO: Using device: cpu
2023-03-28 18:55:27 INFO: Loading: tokenize
2023-03-28 18:55:27 INFO: Loading: pos
2023-03-28 18:55:28 INFO: Loading: lemma
2023-03-28 18:55:28 INFO: Loading: depparse
2023-03-28 18:55:28 INFO: Done loading processors!


['ckip_pos', 'ckip_ner', 'opinion_matcher']

============================= Pipeline Overview =============================

#   Component         Assigns   Requires   Scores   Retokenizes
-   ---------------   -------   --------   ------   -----------
0   ckip_pos                                        False      
                                                               
1   ckip_ner                                        False      
                                                               
2   opinion_matcher                                 False      

✔ No problems found.
{'summary': {'ckip_pos': {'assigns': [], 'requires': [], 'scores': [], 'retokenizes': False}, 'ckip_ner': {'assigns': [], 'requires': [], 'scores': [], 'retokenizes': False}, 'opinion_matcher': {'assigns': [], 'requires': [], 'scores': [], 'retokenizes': False}}, 'problems': {'ckip_pos': [], 'ckip_ner': [], 'opinion_matcher': []}, 'attrs': {}}


In [16]:
doc1 = [ doc for doc in pipeline.pipe(doc1)]
doc2 = [ doc for doc in pipeline.pipe(doc2)]
doc3 = [ doc for doc in pipeline.pipe(doc3)]

In [61]:
for doc in doc1:
    print([(ent, ent.label_) for ent in doc.ents if ent.label_ == 'PERSON'])

[(王世堅, 'PERSON'), (謝龍介, 'PERSON'), (博恩, 'PERSON'), (大爆謝龍介, 'PERSON'), (謝龍介, 'PERSON'), (謝龍介今, 'PERSON'), (博恩, 'PERSON')]
[(謝龍介, 'PERSON'), (博恩, 'PERSON'), (謝龍介, 'PERSON')]
[(謝龍介上, 'PERSON'), (王世堅, 'PERSON'), (博恩, 'PERSON'), (謝龍介, 'PERSON'), (謝龍介, 'PERSON')]
[]
[]


[None, None, None, None, None]


In [129]:
target_span = None
target_doc = None
def run():
    for paragraph_index, doc in enumerate(doc3):
        if 'opinion_found' in doc.spans:
            for i, span in enumerate(doc.spans['opinion_found']):
                if span.text == "他":
                    target_span = span
                    target_doc = doc
                    return paragraph_index 
                print(doc._.paragraph_index, i, span.text, span.label_)
paragraph_index = run()
paragraph_index, target_span, target_doc

(0, None, None)

(1, None, None)

In [132]:
target_span = None
target_doc = None
def run():
    for paragraph_index, doc in enumerate(doc3):
        if 'opinion_found' in doc.spans:
            for i, span in enumerate(doc.spans['opinion_found']):
                if span.text == "他":
                    return paragraph_index, span, doc
                if(span.label_ == "OPINION_SRC_found"):
                    print(i, span.text, span.label_, [t.pos_ for t in span], check_SRC_type(span))

paragraph_index, target_span, target_doc = run()
paragraph_index, target_span, target_doc

(0,
 他,
 〔記者鍾麗華／台北報導〕行政院長蘇貞昌今天（12日）接見立陶宛國會國安及國防事務議員訪團，他致詞時表示，中國的威權擴張毫無道理且蠻橫，台灣應與立陶宛加強合作，一起努力對抗集權擴張行為。)

In [138]:
find_pronoun_resolution(target_doc, paragraph_index, target_span)

span_type ('PRON', '他')
span_type[0] PRON


In [32]:
for doc in doc3:
    if 'opinion_found' in doc.spans:
        for i, span in enumerate(doc.spans['opinion_found']):
            # if span.text == "他":
            #     target_span = span
            #     target_doc = doc
            #     break
            print(doc._.paragraph_index, i, span.text, span.label_)

None 0 他 OPINION_SRC_found
None 1 表示 OPINION_OPR_found
None 2 中國的威權擴張毫無道理且蠻橫 OPINION_SEG_found
None 3 ，台灣應與立陶宛加強合作，一起努力對抗集權擴張行為 OPINION_SEG_found
None 0 蘇揆 OPINION_SRC_found
None 1 強調 OPINION_OPR_found
None 2 立陶宛與台灣都在威權擴張侵略的最前線，兩國同樣堅持自由、民主，有相同的價值 OPINION_SEG_found
None 0 蘇揆 OPINION_SRC_found
None 1 指出 OPINION_OPR_found
None 2 威權擴張侵略時常違反世界期待，如俄羅斯侵略烏克蘭，透過武力遂行擴張目的，除讓民眾變成難民，也影響整個世界的經濟活動；而中國對台灣的軍事騷擾及各種假訊息滲透，也都是違背世界期待、侵擾區域和平 OPINION_SEG_found
None 0 卡斯楚那斯 OPINION_SRC_found
None 1 表示 OPINION_OPR_found
None 2 ，這次與台灣官員、智庫交流有個共識，就是國防不能只靠軍隊，應該要靠社會一起執行全民國防 OPINION_SEG_found
None 3 ，他也願意分享立陶宛的經驗 OPINION_SEG_found
None 0 立陶宛國會友台小組副主席莎卡琳恩 OPINION_SRC_found
None 1 指出 OPINION_OPR_found
None 2 台灣與立陶宛都距離敵人很近 OPINION_SEG_found
None 3 這同時是威脅也是優勢 OPINION_SEG_found
None 4 立陶宛 OPINION_SRC_found
None 5 倡導 OPINION_OPR_found
None 6 如同俄羅斯侵略烏克蘭一樣 OPINION_SEG_found


In [25]:
target_doc
target_span